# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

ModuleNotFoundError: No module named 'gmaps'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
city_data_df = pd.read_csv('cities.csv')
city_data_df.head()

NameError: name 'pd' is not defined

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Configure gmaps
gmaps.configure(api_key=g_key)

NameError: name 'gmaps' is not defined

In [ ]:
# Store latitude and longitude in locations
locations = city_data_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
humidity = city_data_df["Humidity"]

# Plot Heatmap
fig = gmaps.Map()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

# Add layer
fig.add_layer(heat_layer)

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
ideal_city_data_df = city_data_df.loc[city_data_df['Max Temp'] < 80, :]
ideal_city_data_df = ideal_city_data_df.loc[ideal_city_data_df['Max Temp'] > 70, :]
ideal_city_data_df = ideal_city_data_df.loc[ideal_city_data_df['Wind Speed'] < 10, :]
ideal_city_data_df = ideal_city_data_df.loc[ideal_city_data_df['Cloudiness'] == 0, :]
ideal_city_data_df.head()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = ideal_city_data_df
hotel_df['Hotel Name'] = np.nan

target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

idx = 0
for index, row in hotel_df.iterrows():
    # set up a parameters dictionary
    params = {
        "location": f"{row['Lat']}, {row['Lng']}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json['results']
    if len(results) > 0:
        name = response_json['results'][0]['name']
        hotel_df.iloc[idx, -1] = name
    idx = idx + 1

narrowed_city_df = hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

NameError: name 'gmaps' is not defined